In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests
import os
token = os.getenv('GITHUB_TOKEN', '')
import lex, lexxml, lexpy, lexjs
lexpy = lexpy.gen(lex)

In [6]:
project_url = 'https://github.com/conda-forge/libtvm-feedstock'
url_params = '/'.join(project_url.split('/')[-2:])
file_path = 'build-locally.py'
# file_path = '/main/build-locally.py'
query_url = f"https://api.github.com/repos/"+url_params+"/git/blobs/"+file_path
params = {
    "state": "open",
}
headers = {'Authorization': f'token {token}', 'Accept': 'application/vnd.github.diff',}
r = requests.get(query_url, headers=headers, params=params)
diff_list = list(r.iter_lines())

In [2]:
file_dict = pd.read_pickle('Q3_file_dict.pickle')

In [3]:
from urllib.request import urlopen

result_dict = {}
with tqdm(total=len(list(file_dict.keys()))) as pbar:

    for url, files in file_dict.items():
        project_url = url
        url_params = '/'.join(project_url.split('/')[-2:])
        result_dict[url] = {}
        for file_path in files:
            # file_path = 'build-locally.py'
            result_dict[url][file_path] = ''
            try:
                link = "https://raw.githubusercontent.com/"+url_params+"/main/"+file_path
                # print (link)
                f = urlopen(link)
            except:
                try:
                    link = "https://raw.githubusercontent.com/"+url_params+"/master/"+file_path
                    f = urlopen(link)
                except:
                    continue
                
            myfile = f.read()
            result_dict[url][file_path] = myfile.decode('utf-8', 'ignore')
        pbar.update(1)

100%|█████████████████████████████████████████| 835/835 [16:30<00:00,  1.19s/it]


In [11]:
def print_py_token(input_string):
    '''
    param:
        input_string: python source codes
    return:
        
    '''
    lexer = lex.Lexer(lexpy, input_string)
    token_list = []
    while (True):
        t = lexer.get_token();
        if (t is None): 
            break;
        token_list.append(lexer.repr_token(t))
    return token_list

In [13]:
token_list = []

In [50]:
count = 0
for url, temp_dict in result_dict.items():
    for file, data in temp_dict.items():
        if data != '' and count < 665:
            count += 1
            token_list.extend(print_py_token(data))

In [15]:
def token_to_dict(token_string):
    text_string = token_string.split('Token(text=', 1)[1].split(', type=')[0]
    type_string = token_string.split(', type=', 1)[1].split(', flags=')[0]
    flags_string = token_string.split(', flags=', 1)[1].split(', state=')[0]
    state_string = token_string.split(', state=', 1)[1].split(')')[0]

    return text_string, type_string, flags_string, state_string

In [51]:
df_data_list = []
for token_string in token_list:
    df_data_list.append(token_to_dict(token_string))
    # break

In [52]:
# df_data_list

df = pd.DataFrame.from_records(df_data_list, columns=['text', 'type', 'flags', 'state'])
df

,text,type,flags,state
0,'import',KEYWORD,NONE,DEFAULT
1,' ',WHITESPACE,IGNORE,DEFAULT
2,'time',IDENTIFIER,NEXT_NO_OP_PREFIX,DEFAULT
3,'\n',WHITESPACE,IGNORE,DEFAULT
4,'from',KEYWORD,NONE,DEFAULT
...,...,...,...,...
3781154,'.',OPERATOR,NEXT_IS_MEMBER | NEXT_NO_OP_PREFIX,DEFAULT
3781155,'print',IDENTIFIER,MEMBER | NEXT_NO_OP_PREFIX,DEFAULT
3781156,'(',OPERATOR,BRACKET,DEFAULT
3781157,')',OPERATOR,NEXT_NO_OP_PREFIX | BRACKET | BRACKET_CLOSE,DEFAULT


In [55]:
df['text'].nunique()

126738

In [61]:
df[df['type'] == 'KEYWORD']['text'].value_counts()

'if'          25620
'def'         19541
'return'      14883
'in'          13318
'import'      11333
'for'          9020
'else'         6622
'not'          6334
'from'         5665
'and'          3577
'is'           3214
'elif'         3036
'try'          2812
'class'        2777
'except'       2763
'as'           2172
'raise'        2150
'or'           2131
'assert'       1591
'with'         1394
'pass'         1040
'del'           868
'continue'      832
'lambda'        764
'while'         677
'global'        495
'break'         463
'yield'         304
'finally'       185
'nonlocal'        7
Name: text, dtype: int64

In [62]:
df.to_pickle('normal_python_token.py')